In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pyodbc
import urllib
import joblib
import json
from sqlalchemy import create_engine 
from datetime import datetime, date
from datetime import datetime
import pandas as pd
import numpy as np
import os
import joblib
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.__version__

'2.1.4'

In [4]:
import sklearn
sklearn.__version__,\
joblib.__version__,\
xgb.__version__

('1.3.2', '1.3.2', '2.0.3')

In [5]:
# x = pd.read_csv('Users/andrew/Loonie_V5_IBV/traininginputdata.csv')
# y = pd.read_csv('Users/andrew/Loonie_V5_IBV/trainingoutputdata.csv')

x = pd.read_csv('traininginputdata.csv')
y = pd.read_csv('trainingoutputdata.csv')

try:
    del x['Unnamed: 0']
except:
    pass
try:    
    del y['Unnamed: 0']
except:
    pass

In [5]:
x

,ScoredAppID,CustomerID,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,DEBIT_AMOUNT_Z,DEBIT_COUNT_Z,HIGHEST_PAY_DEPOSIT_MEAN,HIGHEST_PAY_FREQUENCY,INCOME_SOURCES_COUNT,MONTH_INFLOW_MEAN,MONTH_INFLOW_SD,MONTH_OUTFLOW_MEAN,MONTH_OUTFLOW_SD,MONTHS_WITH_FEES_RATE,MONTHS_WITH_EMPLOYMENT_RATE,NO_ACTIVITY_RATE,OD_AND_NSF_FEES_DAILY,RECURRENT_COUNT,RECURRING_RATE,BALANCE_MEAN,BALANCE_SD,BALANCE_ABOVE_RATE,LAST_BALANCE_ABOVE,AVG_BALANCE_ABOVE_7D,HIGHER_BALANCE_7D,NUM_LOAN_PMT,TOTAL_LOAN_PMT_AMT,NUM_LOAN_ORIG,TOTAL_LOAN_ORIG_AMT,PAYROLL_AMOUNT,TOTAL_CREDIT,TOTAL_DEBIT,NUM_STOP_PMT,NUM_NSF,NUM_OD,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree,FPDAA,ApplicationDate,AppID,Age
0,10007,5205.0,-0.087015,-0.022488,-0.019994,0.817035,1.149418,0.838944,0.552019,1.160405,1.103358,-0.210191,0.0,1.110430,1.867554,1.167702,1.692363,1.061575,0.0,0.365717,-0.015012,-0.118051,-0.110489,-0.121312,0.037039,-0.631242,-0.682389,-1.099844,-0.806652,0.251003,2.662364,-0.431410,-0.001962,1.010339,0.982637,1.037200,-0.170019,-0.255575,0.881793,0.358335,-0.169955,-0.093519,1.092111,0.982637,1.219218,0.0,2023-11-29 12:17:28,10007,46.0
1,10007,5205.0,-0.087015,-0.022488,-0.019994,0.817035,1.149418,0.838944,0.552019,1.160405,1.103358,-0.210191,0.0,1.110430,1.867554,1.167702,1.692363,1.061575,0.0,0.365717,-0.015012,-0.118051,-0.110489,-0.121312,0.037039,-0.631242,-0.682389,-1.099844,-0.806652,0.251003,2.662364,-0.431410,-0.001962,1.010339,0.982637,1.037200,-0.170019,-0.255575,0.881793,0.358335,-0.169955,-0.093519,1.092111,0.982637,1.219218,1.0,2024-07-16 15:10:26,24119,47.0
2,10007,5205.0,-0.087015,-0.022488,-0.019994,0.817035,1.149418,0.838944,0.552019,1.160405,1.103358,-0.210191,0.0,1.110430,1.867554,1.167702,1.692363,1.061575,0.0,0.365717,-0.015012,-0.118051,-0.110489,-0.121312,0.037039,-0.631242,-0.682389,-1.099844,-0.806652,0.251003,2.662364,-0.431410,-0.001962,1.010339,0.982637,1.037200,-0.170019,-0.255575,0.881793,0.358335,-0.169955,-0.093519,1.092111,0.982637,1.219218,0.0,2024-11-06 13:12:25,35585,47.0
3,10008,5206.0,-0.552740,-0.022410,-0.019999,13.014661,13.076162,0.820509,0.229468,0.903135,22.508496,-0.757139,0.0,12.538362,27.477371,12.409573,22.119243,-0.711948,0.0,-0.697657,-0.220524,-0.118051,-0.110489,4.266427,36.442410,-0.278179,-0.682389,-1.099844,-0.806652,-0.207692,-0.036216,-0.534191,-0.530908,0.609653,8.168059,8.106577,-0.170019,-0.367781,-0.541902,-0.359436,-0.230528,-0.116733,-0.726466,8.168059,-0.969528,0.0,2023-11-29 12:28:13,10008,47.0
4,10022,5212.0,-0.127513,-0.022348,-0.020091,0.298721,0.306902,0.517239,1.216106,1.227255,0.013633,-0.289268,0.0,0.372143,0.630680,0.362831,0.603709,0.396504,0.0,-0.291915,-0.007813,-0.118051,-0.110489,-0.013863,-0.247179,0.887913,-0.682389,0.909220,-0.806652,3.278392,0.472367,-0.482801,0.360629,0.360701,0.318008,0.310778,-0.170019,0.305452,-0.304619,0.597593,-0.063952,-0.087664,0.049886,0.318008,0.115361,0.0,2023-11-29 13:33:43,10022,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4633,9922,5164.0,-0.087015,-0.022259,-0.020054,0.363131,0.438489,1.030754,-0.210023,0.395955,0.328501,-0.454011,0.0,0.310335,0.210605,0.291423,0.468625,1.061575,0.0,-0.391325,-0.073185,-0.118051,-0.110489,-0.231605,-0.374942,-1.095429,-0.682389,-1.099844,-0.806652,-0.391170,-0.365087,-0.534191,-0.530908,1.220971,0.262367,0.246330,-0.170019,-0.143370,0.881793,0.119078,-0.230528,-0.116733,-0.357787,0.262367,-0.305697,0.0,2024-03-01 09:46:06,15884,42.0
4634,9922,5164.0,-0.087015,-0.022259,-0.020054,0.363131,0.438489,1.030754,-0.210023,0.395955,0.328501,-0.454011,0.0,0.310335,0.210605,0.291423,0.468625,1.061575,0.0,-0.391325,-0.073185,-0.118051,-0.110489,-0.231605,-0.374942,-1.095429,-0.682389,-1.099844,-0.806652,-0.391170,-0.365087,-0.534191,-0.530908,1.220971,0.26

In [6]:
clf = xgb.XGBClassifier(max_depth=3, random_state=123)
features_modeling = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD', 'DAILY_DEBIT_AMOUNT_MEAN', 
                   'DAILY_INCOME_MEAN', 'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z', 
                   'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY', 'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN',
                   'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                   'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE', 'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 
                   'RECURRING_RATE', 'BALANCE_MEAN', 'BALANCE_SD', 'BALANCE_ABOVE_RATE', 'LAST_BALANCE_ABOVE',
                   'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT', 'NUM_LOAN_ORIG', 
                   'TOTAL_LOAN_ORIG_AMT', 'PAYROLL_AMOUNT', 'TOTAL_CREDIT', 'TOTAL_DEBIT', 'NUM_STOP_PMT', 'NUM_NSF',
                   'NUM_OD', 'NUM_RETURN_PMT', 'NUM_GAMBLING_PMT', 'AMT_GAMBLING_PMT', 'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']
clf.fit(x[features_modeling], y.FPDAA.values)
from sklearn.metrics import roc_curve, auc
def score(val, pred):
    fpr, tpr, thresholds = roc_curve(val, pred[:,1]) 
    roc_auc = auc(fpr, tpr) 
    return roc_auc
pred_dt = clf.predict_proba(x[features_modeling])
print(score(y.FPDAA.values, pred_dt)) 

0.9416333489785755


In [9]:
# joblib.dump(clf,'Users/andrew/Loonie_V3_IBV/IBV_V3_TestBed.pkl')

joblib.dump(clf,'IBV_V3_TestBed.pkl')

['IBV_V3_TestBed.pkl']

In [10]:
y.FPDAA

0       0.0
1       1.0
2       0.0
3       0.0
4       0.0
       ... 
4633    0.0
4634    0.0
4635    0.0
4636    0.0
4637    0.0
Name: FPDAA, Length: 4638, dtype: float64

In [11]:
# df = pd.read_csv('Users/andrew/Loonie_V3_IBV/ibv_input_data.csv')

df = pd.read_csv('ibv_input_data.csv')
df = df.dropna(subset=['MaxHistory'])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_curve, auc
features_modeling = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD', 'DAILY_DEBIT_AMOUNT_MEAN', 
                   'DAILY_INCOME_MEAN', 'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z', 
                   'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY', 'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN',
                   'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                   'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE', 'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 
                   'RECURRING_RATE', 'BALANCE_MEAN', 'BALANCE_SD', 'BALANCE_ABOVE_RATE', 'LAST_BALANCE_ABOVE',
                   'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT', 'NUM_LOAN_ORIG', 
                   'TOTAL_LOAN_ORIG_AMT', 'PAYROLL_AMOUNT', 'TOTAL_CREDIT', 'TOTAL_DEBIT', 'NUM_STOP_PMT', 'NUM_NSF',
                   'NUM_OD', 'NUM_RETURN_PMT', 'NUM_GAMBLING_PMT', 'AMT_GAMBLING_PMT', 'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']

x = df[df.Age>35]
y = df[df.Age>35][['FPDAA']]
# x[features_modeling].to_csv('trainingforstandardization.csv' ,index=False)

scaler = StandardScaler()
scaler.fit(x[features_modeling])

StandardScaler()

In [13]:
scaler

StandardScaler()

In [14]:
# joblib.dump(scaler,'Users/andrew/Loonie_V3_IBV/IBV_SCALER_V3_TestBed.pkl')
# joblib.dump(scaler,'IBV_SCALER_V3_TestBed.pkl')
joblib.dump(scaler,'standardscaler_V3.pkl')

['standardscaler_V3.pkl']

In [7]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import statsmodels.api as sm
import joblib
import json
import sys
import logging

# example: jsonstr = "IBV_1051.json"
# example: modelfilepath = "IBV_V3_TestBed.pkl"
# example: scalerfilepath = 'standardscaler_V3.pkl'
# example use case: ibvmodel(jsonstr, modelfilepath, scalerfilepath)

def ibvmodel(jsonstr, modelfilepath, scalerfilepath):
    
    logging.basicConfig(filename="IBVModel_WarningErrorLog_2023.log", level=logging.WARNING, 
            format="%(asctime)s - %(levelname)s - %(message)s")
    stepname = '1 Loading Json'

    try:
        # Load and Read the JSON file
        json_temp = json.loads(jsonstr)
        # Load Json file's historical transaction part
        df_ibv = pd.read_json(json.dumps(json_temp['Historical_Transactions']))

        # Data Cleaning
        stepname = '2 Data Cleaning'
        df_ibv['IBV_Credit'] = np.where(df_ibv['amount']<0, -1*df_ibv['amount'],0)
        df_ibv['IBV_Debit'] = np.where(df_ibv['amount']>=0, df_ibv['amount'],0)
        df_ibv = df_ibv.rename(columns={'date':'IBV_Date','category':'IBV_BalCategory','name':'IBV_Description'})
        df_ibv['IBV_Date'] = pd.to_datetime(df_ibv['IBV_Date'])
        df_ibv['Year'] = df_ibv['IBV_Date'].apply(lambda x: x.year)
        df_ibv['Month'] = df_ibv['IBV_Date'].apply(lambda x: x.month)
        df_ibv = df_ibv[['account_id','IBV_Credit','IBV_Debit','IBV_Date','Year','Month','IBV_BalCategory','IBV_Description']]
        df_ibv['isLoan'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Service","Financial","Loans and Mortgages"],
                                            ["Travel and Transportation","Auto Loan"], ["Education","Student Loan"],
                                            ["Miscellaneous","Point of Sale Loan"],["Miscellaneous","Insolvency Loan"],
                                            ["Miscellaneous","Other Loan"],["Miscellaneous","Micro Loan"]] else 0)
        df_ibv['regularPayroll'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Transfer","Payroll"], ["Transfer","Payroll",'Benefits'],
                                    ["Income","Pension"],["Income","Government"],["Income","Wages and Salary"]] else 0)
        df_ibv['isFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","ATM"],["Bank Fees","Cash Advance"],
                                                ["Bank Fees","Excess Activity"], ["Bank Fees","Foreign Transaction"],
                                                ["Bank Fees","Fraud Dispute"],["Bank Fees","Insufficient Funds"],
                                                ["Bank Fees","Late Payment"], ["Bank Fees","Overdraft"],
                                                ["Bank Fees","Wire Transfer"],["Bank Fees"],
                                                ["Fees and Charges","ATM Fees"], ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isNSFFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","Insufficient Funds"],["Bank Fees","Overdraft"],
                                            ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isDeposit'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Transfer","Deposit","ATM"],
                                                ["Transfer","Deposit","Check"],["Transfer","Deposit"], ["Miscellaneous Income","Deposit"]]) else 0)
        df_ibv['IBV_BalCategory'] = df_ibv['IBV_BalCategory'].apply(str)
        accountid = df_ibv['account_id'].values[0]

        # Load Json file's historical balance part
        primary_account_info = [i for i in json_temp['Historical_Balance']['report']['items'][0]['accounts'] 
                            if i['account_id']==df_ibv.account_id.values[0]]
        if primary_account_info == []:
            df_balance = pd.DataFrame( [[0.00, '2023-02-06', 'CAD' , None]], columns = ['IBV_Balance','date',
                                                        'iso_currency_code', 'unofficial_currency_code'] )
        else:
            df_balance = pd.DataFrame(primary_account_info[0]['historical_balances'])
            df_balance = df_balance.rename(columns={'current':'IBV_Balance'})
        df_balance.date = pd.to_datetime(df_balance.date)
        
        ### Make sure you change it!!!
        curdate = date.today()
        # curdate = df_balance['date'].max().date() # remove it before deployment
        
        df_balance['within_last_7day'] = df_balance.apply(lambda x: (curdate - x['date'].date()).days<=7, axis=1)

        ## Step 2: Feature Generation for IBV: to get the input of IBV_Model

        # Feature Generation

        # # Max History
        stepname = '3 Feature Generation'
        
        f1 = pd.DataFrame([[df_ibv.account_id.values[0],(df_ibv['IBV_Date'].max() - df_ibv['IBV_Date'].min()).days]],
                      columns = ['account_id','MaxHistory'])

        # # CREDIT_TO_DEBIT_RATIO_MEAN, CREDIT_TO_DEBIT_RATIO_SD
        f2 = df_ibv.groupby(['account_id','Year','Month']).sum()[['IBV_Credit','IBV_Debit']].reset_index()
        f2['CREDIT_TO_DEBIT_RATIO'] = f2['IBV_Credit'] / f2['IBV_Debit']
        f2 = f2.groupby('account_id').agg({'CREDIT_TO_DEBIT_RATIO':['mean','std']}).reset_index()
        f2.columns = ['account_id','CREDIT_TO_DEBIT_RATIO_MEAN','CREDIT_TO_DEBIT_RATIO_SD']

        # # DAILY_DEBIT_AMOUNT_MEAN, DAILY_INCOME_MEAN
        f3 = df_ibv[df_ibv.isLoan==0].groupby(['account_id','Year','Month']).sum()[['IBV_Credit','IBV_Debit']].reset_index()
        f3['IBV_Credit'] = f3['IBV_Credit'] / 30
        f3['IBV_Debit'] = f3['IBV_Debit'] / 30
        f3 = f3.groupby('account_id').mean()[['IBV_Debit','IBV_Credit']].reset_index()
        f3.columns = ['account_id','DAILY_DEBIT_AMOUNT_MEAN','DAILY_INCOME_MEAN']

        # # DAILY_INCOME_REGULAR_MEAN
        f4 = df_ibv[df_ibv.regularPayroll==1].groupby(['account_id','Year','Month']).sum()['IBV_Credit'].reset_index()
        f4['IBV_Credit'] = f4['IBV_Credit'] / 30
        f4 = f4.groupby('account_id').mean()['IBV_Credit'].reset_index()
        f4.columns = ['account_id','DAILY_INCOME_REGULAR_MEAN']

        # # DEBIT_AMOUNT_AVG, DEBIT_AMOUNT_SD, DEBIT_COUNT_AVG,DEBIT_COUNT_SD
        f5 = df_ibv.groupby(['account_id','Year','Month']).sum()['IBV_Debit'].reset_index()
        f5 = f5.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f5.columns = ['account_id','DEBIT_AMOUNT_MEAN','DEBIT_AMOUNT_SD']

        f6 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month']).count()['IBV_Debit'].reset_index()
        f6 = f6.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f6.columns = ['account_id','DEBIT_COUNT_MEAN','DEBIT_COUNT_SD']

        # # DEBIT_AMOUNT_Z, DEBIT_COUNT_Z
        f71 = df_ibv.groupby(['account_id','Year','Month']).sum()['IBV_Debit'].reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f71 = f71.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f71 = f71.merge(f5,on='account_id')
        f71['DEBIT_AMOUNT_Z'] = (f71['IBV_Debit'] - f71['DEBIT_AMOUNT_MEAN'])/f71['DEBIT_AMOUNT_SD']

        f72 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month']).count()['IBV_Debit'].reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f72 = f72.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f72 = f72.merge(f6,on='account_id')
        f72['DEBIT_COUNT_Z'] = (f72['IBV_Debit'] - f72['DEBIT_COUNT_MEAN'])/f72['DEBIT_COUNT_SD']

        f7 = f71[['account_id', 'DEBIT_AMOUNT_Z']].merge(f72[['account_id','DEBIT_COUNT_Z']], on='account_id', how='left')


        # # HIGHEST_PAY_DEPOSIT_MEAN, HIGHEST_PAY_FREQUENCY
        f81 = df_ibv.groupby(['account_id','IBV_BalCategory','Year','Month']).sum()['IBV_Credit'].reset_index()
        f81 = f81.groupby(['account_id','IBV_BalCategory']).mean()['IBV_Credit'].reset_index().sort_values(by=['account_id','IBV_Credit','IBV_BalCategory'],ascending=False)
        f82 = f81.groupby('account_id').first().reset_index()[['account_id','IBV_BalCategory']]
        f8 = f81.merge(f82, on =['account_id','IBV_BalCategory'], how='inner')[['account_id','IBV_Credit']]
        f8.columns = ['account_id','HIGHEST_PAY_DEPOSIT_MEAN']

        f9 = df_ibv[df_ibv.IBV_Credit>0].merge(f82, on =['account_id','IBV_BalCategory'], how='inner').groupby(['account_id','Year','Month']).count()['IBV_Credit'].reset_index()
        f9 = f9.groupby('account_id')['IBV_Credit'].mean().reset_index()[['account_id','IBV_Credit']]
        f9.columns = ['account_id','HIGHEST_PAY_FREQUENCY']

        # # INCOME_SOURCES_COUNT
        f10 = df_ibv[(df_ibv.IBV_Credit>0)&(df_ibv.IBV_BalCategory.isin(['income/investment_income','income/paycheck','income/bonus','income/government',
        'income','income/pension','income/child_support']))].groupby(['account_id','IBV_BalCategory']).size().reset_index()
        f10 = f10.groupby('account_id').size().reset_index()
        f10.columns = ['account_id','INCOME_SOURCES_COUNT']



        # # MONTH_INFLOW_MEAN, MONTH_INFLOW_SD, MONTH_OUTFLOW_MEAN, MONTH_OUTFLOW_SD
        f11 = df_ibv.groupby(['account_id','Year','Month']).sum()[['IBV_Credit','IBV_Debit']].reset_index()
        f11 = f11.groupby('account_id').agg({'IBV_Credit':['mean','std'],'IBV_Debit':['mean','std']}).reset_index()
        f11.columns = ['account_id','MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD']


        # #MONTHS_WITH_FEES_RATE
        f12_ = df_ibv.copy()
        f12 = f12_.groupby(['account_id','Year','Month']).sum()['isFees'].reset_index()
        f12['MONTHS_WITH_FEES_RATE'] = np.where(f12['isFees']>0,1.0,0.0)
        f12 = f12.groupby('account_id').mean()['MONTHS_WITH_FEES_RATE'].reset_index()[['account_id','MONTHS_WITH_FEES_RATE']]

        # # MONTHS_WITH_EMPLOYMENT_RATE
        f12_['Emp'] = (df_ibv['IBV_BalCategory']=='income/paycheck').astype(int) #!!!!!
        f13 = f12_.groupby(['account_id','Year','Month']).sum()['Emp'].reset_index()
        f13['MONTHS_WITH_EMPLOYMENT_RATE'] = np.where(f13['Emp']>0,1.0,0.0)
        f13 = f13.groupby('account_id').mean()['MONTHS_WITH_EMPLOYMENT_RATE'].reset_index()[['account_id','MONTHS_WITH_EMPLOYMENT_RATE']]

        # # NO_ACTIVITY_RATE
        f14 = df_ibv.groupby('account_id')['IBV_Date'].nunique().reset_index()
        f14 = f14.merge(f1, on='account_id')[['account_id','IBV_Date','MaxHistory']]
        f14['NO_ACTIVITY_RATE'] = f14['IBV_Date'] / f14['MaxHistory']
        f14 = f14[['account_id','NO_ACTIVITY_RATE']]

        # # OD_AND_NSF_FEES_DAILY
        f15 = df_ibv[df_ibv.isNSFFees==1].groupby('account_id')['IBV_Debit'].sum().reset_index()
        f15 = f15.merge(f1, on='account_id')[['account_id','IBV_Debit','MaxHistory']]
        f15['OD_AND_NSF_FEES_DAILY'] = f15['IBV_Debit'] / f15['MaxHistory']
        f15 = f15[['account_id','OD_AND_NSF_FEES_DAILY']]

        # # RECURRENT_COUNT, RECURRENT_RATE
        f16 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id', 'Year', 'Month', 'IBV_Debit', 'IBV_BalCategory']).size()
        f16 = f16.groupby(['account_id','IBV_Debit','IBV_BalCategory']).size().reset_index()
        f16.columns = ['account_id','IBV_Debit', 'IBV_BalCategory', 'CNT']
        f16 = f16[(f16['CNT']>=5)&(f16['IBV_Debit']>=10)]
        f16 = f16.groupby('account_id').agg({'IBV_BalCategory':'nunique','IBV_Debit':'sum'}).reset_index()
        f16.columns = ['account_id','RECURRENT_COUNT','RECURRING_RATE']
        
        f17 = pd.DataFrame([[accountid, df_balance['IBV_Balance'].mean(), df_balance['IBV_Balance'].std(),
                            (df_balance['IBV_Balance']>200).mean(), int(df_balance['IBV_Balance'].values[0]>200),
                            int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].values.mean())>200),
             int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].mean())>(df_balance[df_balance.within_last_7day==False]['IBV_Balance'].mean())) ]], 
              columns = ['account_id','BALANCE_MEAN','BALANCE_SD','BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                         'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D'])
        
        # # Red Zone features added by Coral: loan, payroll, deposit, total credit, total debit
        f18 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Debit']>60)].groupby('account_id').agg({'IBV_Debit':['count','sum']}).reset_index()
        f18.columns = ['account_id', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT']
        
        f19 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Credit']>60)].groupby('account_id').agg({'IBV_Credit':['count','sum']}).reset_index()
        f19.columns = ['account_id', 'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT']
        
        f20 = df_ibv[(df_ibv['regularPayroll']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f20.columns = ['account_id','PAYROLL_AMOUNT']
        

        f21 = df_ibv[(df_ibv['isDeposit']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f21.columns = ['account_id','DEPOSIT_AMOUNT']
        f22 = df_ibv.groupby('account_id')[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f22.columns = ['account_id','TOTAL_CREDIT','TOTAL_DEBIT']
        
        # # Alert indicators added by Coral: stop payment, nsf, overdraft, return
        stop_keywords = ['e-transfer stop', 'stop payt','stop payment','stop fee','stop pmt','stoppa','stop pymt']
        nsf_keywords = [r"\bnsf", 'NON-SUFFICIENT FUNDS','Insufficient fund','NON SUFFICIENT','Returned Item Fee']
        overdraft_keywords = ['Overdraft', 'Over limit','OD FEE','OD PROTECTION','OD HANDL']
        return_keywords = ['returned', 'RETURN FEE','Return of','EFT.*Return','EFT.*Reversal','rtn.*eft']
        gambling_keywords = ['payper', 'paybilt',' gigadat', 'playnow', 'bet.*river','betmgm','betty.*gaming',
                            'bally.*bet', 'pointsbet', 'bet365', 'betway', 'betano', 'thescore.*bet', 'northstar.*bet',
                            'lotto', 'playalberta', 'playtime', 'casino', 'ilixium.*casin']
        
        df_ibv['STOP_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(stop_keywords), case=False)) & (df_ibv['IBV_Credit']>0), 'STOP_Incident'] = 1
        df_ibv['NSF_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(nsf_keywords), case=False)), 'NSF_Incident'] = 1
        df_ibv['OVERDRAFT_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(overdraft_keywords), case=False)), 'OVERDRAFT_Incident'] = 1
        df_ibv['RETURN_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(return_keywords), case=False)), 'RETURN_Incident'] = 1
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'Gambling_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'IBV_Debit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'EI_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'IBV_Credit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'bree'), case=False)), 'BREE_Incident'] = 1
        
        
        f23 = df_ibv.groupby('account_id').agg(NUM_STOP_PMT = ('STOP_Incident','sum'),
                                               NUM_NSF = ('NSF_Incident','sum'),
                                               NUM_OD = ('OVERDRAFT_Incident','sum'),
                                               NUM_RETURN_PMT = ('RETURN_Incident','sum'),
                                               NUM_GAMBLING_PMT = ('Gambling_Incident', 'count'),
                                               AMT_GAMBLING_PMT = ('Gambling_Incident', 'sum'),
                                               NUM_EI_PMT = ('EI_Incident', 'count'),
                                               AMT_EI_PMT = ('EI_Incident', 'sum'),
                                               NUM_Bree = ('BREE_Incident', 'count')
                                              ).reset_index()

        
        ibv_modelinput = f1.copy()
        for i in [f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23]:
            ibv_modelinput = ibv_modelinput.merge(i, on='account_id',how='left')

        # ## Step 3: DataCleaning after feature generation
        stepname = '4 DataCleaning after Feature Gen'
        ibv_modelinput = ibv_modelinput.fillna(value=0).replace(np.inf, 100).replace(-np.inf, -100)
        # ## Step 4: IBV Model Scoring and IBVBand Assignment based on IBVScore
        ibv_features = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
                'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
                'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
                'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
                'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
                'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
                'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
                'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
                'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
                'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT',
                'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']
        
        # Load Model
        stepname = '5 Model Loading & Scoring'
        scaler = joblib.load(scalerfilepath)
        clf_ibv = joblib.load(modelfilepath)

        # Get IBV Score (output 1)
        standardized_data = ibv_modelinput[ibv_features].copy()
        # return ibv_modelinput, scaler, standardized_data,clf_ibv
        standardized_data[ibv_features] = scaler.transform(standardized_data)
        ibv_modelinput['IBVScore'] = 1000 - np.round(clf_ibv.predict_proba(standardized_data)[0][1]*1000,0).astype(int)

        # Get IBV Band (output 2)
        ibv_modelinput['IBVBand'] = np.where(ibv_modelinput['IBVScore']< 452,1, 
                                             np.where(ibv_modelinput['IBVScore']< 621,2, 
                                            np.where(ibv_modelinput['IBVScore'] < 700,3,
                                            np.where(ibv_modelinput['IBVScore'] < 753,4,
                                            5))))
        IBVScore = int(ibv_modelinput.IBVScore.values[0])
        IBVBand = int(ibv_modelinput.IBVBand.values[0])


    except:
        try:
            bankaccountid = df_ibv['account_id'].values[0]
            ndbaccountid = json_temp['NDB']['accountnumber']
            logging.exception('AccountID: ' + str(bankaccountid) + '; NDBAccount: ' + str(ndbaccountid) + '; ErrorInStep: ' + stepname )
        except: # If plaid file received is empty: account_id will be 0, or modified to Other value 
            logging.exception('AccountID: ' + 'Not Available' + '; ErrorInStep: ' + stepname )
        finally:
            return "{\"ModelScore\":"+str(0)+ ",\"IBVBand\":" + str(0)+"}"

 
    return "{\"ModelScore\":"+str(IBVScore)+ ",\"IBVBand\":" + str(IBVBand)+"}"

In [8]:
len(features_modeling)

43

In [9]:
ibv_features = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
                'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
                'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
                'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
                'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
                'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
                'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
                'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
                'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
                'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT',
                'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']

In [10]:
len(ibv_features)

43

In [11]:
ibv_features == features_modeling

True

In [12]:
def ibvmodel(jsonstr, modelfilepath, scalerfilepath):
    
    logging.basicConfig(filename="IBVModel_WarningErrorLog_2023.log", level=logging.WARNING, 
            format="%(asctime)s - %(levelname)s - %(message)s")
    stepname = '1 Loading Json'
    if True:
    # try:
        # Load and Read the JSON file  #### This is for testing changes made different from production
        # json_temp = json.loads(jsonstr)
        with open(jsonstr) as json_file:
            json_temp = json.load(json_file)
            

        # Load Json file's historical transaction part
        df_ibv = pd.read_json(json.dumps(json_temp['Historical_Transactions']))

        # Data Cleaning
        stepname = '2 Data Cleaning'
        df_ibv['IBV_Credit'] = np.where(df_ibv['amount']<0, -1*df_ibv['amount'],0)
        df_ibv['IBV_Debit'] = np.where(df_ibv['amount']>=0, df_ibv['amount'],0)
        df_ibv = df_ibv.rename(columns={'date':'IBV_Date','category':'IBV_BalCategory','name':'IBV_Description'})
        df_ibv['IBV_Date'] = pd.to_datetime(df_ibv['IBV_Date'])
        df_ibv['Year'] = df_ibv['IBV_Date'].apply(lambda x: x.year)
        df_ibv['Month'] = df_ibv['IBV_Date'].apply(lambda x: x.month)
        df_ibv = df_ibv[['account_id','IBV_Credit','IBV_Debit','IBV_Date','Year','Month','IBV_BalCategory','IBV_Description']]
        df_ibv['isLoan'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Service","Financial","Loans and Mortgages"],
                                            ["Travel and Transportation","Auto Loan"], ["Education","Student Loan"],
                                            ["Miscellaneous","Point of Sale Loan"],["Miscellaneous","Insolvency Loan"],
                                            ["Miscellaneous","Other Loan"],["Miscellaneous","Micro Loan"]] else 0)
        df_ibv['regularPayroll'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Transfer","Payroll"], ["Transfer","Payroll",'Benefits'],
                                    ["Income","Pension"],["Income","Government"],["Income","Wages and Salary"]] else 0)
        df_ibv['isFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","ATM"],["Bank Fees","Cash Advance"],
                                                ["Bank Fees","Excess Activity"], ["Bank Fees","Foreign Transaction"],
                                                ["Bank Fees","Fraud Dispute"],["Bank Fees","Insufficient Funds"],
                                                ["Bank Fees","Late Payment"], ["Bank Fees","Overdraft"],
                                                ["Bank Fees","Wire Transfer"],["Bank Fees"],
                                                ["Fees and Charges","ATM Fees"], ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isNSFFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","Insufficient Funds"],["Bank Fees","Overdraft"],
                                            ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isDeposit'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Transfer","Deposit","ATM"],
                                                ["Transfer","Deposit","Check"],["Transfer","Deposit"], ["Miscellaneous Income","Deposit"]]) else 0)
        df_ibv['IBV_BalCategory'] = df_ibv['IBV_BalCategory'].apply(str)
        accountid = df_ibv['account_id'].values[0]

        # Load Json file's historical balance part
        primary_account_info = [i for i in json_temp['Historical_Balance']['report']['items'][0]['accounts'] 
                            if i['account_id']==df_ibv.account_id.values[0]]
        if primary_account_info == []:
            df_balance = pd.DataFrame( [[0.00, '2023-02-06', 'CAD' , None]], columns = ['IBV_Balance','date',
                                                        'iso_currency_code', 'unofficial_currency_code'] )
        else:
            df_balance = pd.DataFrame(primary_account_info[0]['historical_balances'])
            df_balance = df_balance.rename(columns={'current':'IBV_Balance'})
        df_balance.date = pd.to_datetime(df_balance.date)
        
        ### Make sure you change it!!!
        curdate = date.today()
        # curdate = df_balance['date'].max().date() # remove it before deployment
        
        df_balance['within_last_7day'] = df_balance.apply(lambda x: (curdate - x['date'].date()).days<=7, axis=1)

        ## Step 2: Feature Generation for IBV: to get the input of IBV_Model

        # Feature Generation

        # # Max History
        stepname = '3 Feature Generation'
        
        f1 = pd.DataFrame([[df_ibv.account_id.values[0],(df_ibv['IBV_Date'].max() - df_ibv['IBV_Date'].min()).days]],
                      columns = ['account_id','MaxHistory'])

        # # CREDIT_TO_DEBIT_RATIO_MEAN, CREDIT_TO_DEBIT_RATIO_SD
        f2 = df_ibv.groupby(['account_id','Year','Month']).sum()[['IBV_Credit','IBV_Debit']].reset_index()
        f2['CREDIT_TO_DEBIT_RATIO'] = f2['IBV_Credit'] / f2['IBV_Debit']
        f2 = f2.groupby('account_id').agg({'CREDIT_TO_DEBIT_RATIO':['mean','std']}).reset_index()
        f2.columns = ['account_id','CREDIT_TO_DEBIT_RATIO_MEAN','CREDIT_TO_DEBIT_RATIO_SD']

        # # DAILY_DEBIT_AMOUNT_MEAN, DAILY_INCOME_MEAN
        f3 = df_ibv[df_ibv.isLoan==0].groupby(['account_id','Year','Month']).sum()[['IBV_Credit','IBV_Debit']].reset_index()
        f3['IBV_Credit'] = f3['IBV_Credit'] / 30
        f3['IBV_Debit'] = f3['IBV_Debit'] / 30
        f3 = f3.groupby('account_id').mean()[['IBV_Debit','IBV_Credit']].reset_index()
        f3.columns = ['account_id','DAILY_DEBIT_AMOUNT_MEAN','DAILY_INCOME_MEAN']

        # # DAILY_INCOME_REGULAR_MEAN
        f4 = df_ibv[df_ibv.regularPayroll==1].groupby(['account_id','Year','Month']).sum()['IBV_Credit'].reset_index()
        f4['IBV_Credit'] = f4['IBV_Credit'] / 30
        f4 = f4.groupby('account_id').mean()['IBV_Credit'].reset_index()
        f4.columns = ['account_id','DAILY_INCOME_REGULAR_MEAN']

        # # DEBIT_AMOUNT_AVG, DEBIT_AMOUNT_SD, DEBIT_COUNT_AVG,DEBIT_COUNT_SD
        f5 = df_ibv.groupby(['account_id','Year','Month']).sum()['IBV_Debit'].reset_index()
        f5 = f5.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f5.columns = ['account_id','DEBIT_AMOUNT_MEAN','DEBIT_AMOUNT_SD']

        f6 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month']).count()['IBV_Debit'].reset_index()
        f6 = f6.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f6.columns = ['account_id','DEBIT_COUNT_MEAN','DEBIT_COUNT_SD']

        # # DEBIT_AMOUNT_Z, DEBIT_COUNT_Z
        f71 = df_ibv.groupby(['account_id','Year','Month']).sum()['IBV_Debit'].reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f71 = f71.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f71 = f71.merge(f5,on='account_id')
        f71['DEBIT_AMOUNT_Z'] = (f71['IBV_Debit'] - f71['DEBIT_AMOUNT_MEAN'])/f71['DEBIT_AMOUNT_SD']

        f72 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month']).count()['IBV_Debit'].reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f72 = f72.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f72 = f72.merge(f6,on='account_id')
        f72['DEBIT_COUNT_Z'] = (f72['IBV_Debit'] - f72['DEBIT_COUNT_MEAN'])/f72['DEBIT_COUNT_SD']

        f7 = f71[['account_id', 'DEBIT_AMOUNT_Z']].merge(f72[['account_id','DEBIT_COUNT_Z']], on='account_id', how='left')


        # # HIGHEST_PAY_DEPOSIT_MEAN, HIGHEST_PAY_FREQUENCY
        f81 = df_ibv.groupby(['account_id','IBV_BalCategory','Year','Month']).sum()['IBV_Credit'].reset_index()
        f81 = f81.groupby(['account_id','IBV_BalCategory']).mean()['IBV_Credit'].reset_index().sort_values(by=['account_id','IBV_Credit','IBV_BalCategory'],ascending=False)
        f82 = f81.groupby('account_id').first().reset_index()[['account_id','IBV_BalCategory']]
        f8 = f81.merge(f82, on =['account_id','IBV_BalCategory'], how='inner')[['account_id','IBV_Credit']]
        f8.columns = ['account_id','HIGHEST_PAY_DEPOSIT_MEAN']

        f9 = df_ibv[df_ibv.IBV_Credit>0].merge(f82, on =['account_id','IBV_BalCategory'], how='inner').groupby(['account_id','Year','Month']).count()['IBV_Credit'].reset_index()
        f9 = f9.groupby('account_id')['IBV_Credit'].mean().reset_index()[['account_id','IBV_Credit']]
        f9.columns = ['account_id','HIGHEST_PAY_FREQUENCY']

        # # INCOME_SOURCES_COUNT
        f10 = df_ibv[(df_ibv.IBV_Credit>0)&(df_ibv.IBV_BalCategory.isin(['income/investment_income','income/paycheck','income/bonus','income/government',
        'income','income/pension','income/child_support']))].groupby(['account_id','IBV_BalCategory']).size().reset_index()
        f10 = f10.groupby('account_id').size().reset_index()
        f10.columns = ['account_id','INCOME_SOURCES_COUNT']



        # # MONTH_INFLOW_MEAN, MONTH_INFLOW_SD, MONTH_OUTFLOW_MEAN, MONTH_OUTFLOW_SD
        f11 = df_ibv.groupby(['account_id','Year','Month']).sum()[['IBV_Credit','IBV_Debit']].reset_index()
        f11 = f11.groupby('account_id').agg({'IBV_Credit':['mean','std'],'IBV_Debit':['mean','std']}).reset_index()
        f11.columns = ['account_id','MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD']


        # #MONTHS_WITH_FEES_RATE
        f12_ = df_ibv.copy()
        f12 = f12_.groupby(['account_id','Year','Month']).sum()['isFees'].reset_index()
        f12['MONTHS_WITH_FEES_RATE'] = np.where(f12['isFees']>0,1.0,0.0)
        f12 = f12.groupby('account_id').mean()['MONTHS_WITH_FEES_RATE'].reset_index()[['account_id','MONTHS_WITH_FEES_RATE']]

        # # MONTHS_WITH_EMPLOYMENT_RATE
        f12_['Emp'] = (df_ibv['IBV_BalCategory']=='income/paycheck').astype(int) #!!!!!
        f13 = f12_.groupby(['account_id','Year','Month']).sum()['Emp'].reset_index()
        f13['MONTHS_WITH_EMPLOYMENT_RATE'] = np.where(f13['Emp']>0,1.0,0.0)
        f13 = f13.groupby('account_id').mean()['MONTHS_WITH_EMPLOYMENT_RATE'].reset_index()[['account_id','MONTHS_WITH_EMPLOYMENT_RATE']]

        # # NO_ACTIVITY_RATE
        f14 = df_ibv.groupby('account_id')['IBV_Date'].nunique().reset_index()
        f14 = f14.merge(f1, on='account_id')[['account_id','IBV_Date','MaxHistory']]
        f14['NO_ACTIVITY_RATE'] = f14['IBV_Date'] / f14['MaxHistory']
        f14 = f14[['account_id','NO_ACTIVITY_RATE']]

        # # OD_AND_NSF_FEES_DAILY
        f15 = df_ibv[df_ibv.isNSFFees==1].groupby('account_id')['IBV_Debit'].sum().reset_index()
        f15 = f15.merge(f1, on='account_id')[['account_id','IBV_Debit','MaxHistory']]
        f15['OD_AND_NSF_FEES_DAILY'] = f15['IBV_Debit'] / f15['MaxHistory']
        f15 = f15[['account_id','OD_AND_NSF_FEES_DAILY']]

        # # RECURRENT_COUNT, RECURRENT_RATE
        f16 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id', 'Year', 'Month', 'IBV_Debit', 'IBV_BalCategory']).size()
        f16 = f16.groupby(['account_id','IBV_Debit','IBV_BalCategory']).size().reset_index()
        f16.columns = ['account_id','IBV_Debit', 'IBV_BalCategory', 'CNT']
        f16 = f16[(f16['CNT']>=5)&(f16['IBV_Debit']>=10)]
        f16 = f16.groupby('account_id').agg({'IBV_BalCategory':'nunique','IBV_Debit':'sum'}).reset_index()
        f16.columns = ['account_id','RECURRENT_COUNT','RECURRING_RATE']
        
        f17 = pd.DataFrame([[accountid, df_balance['IBV_Balance'].mean(), df_balance['IBV_Balance'].std(),
                            (df_balance['IBV_Balance']>200).mean(), int(df_balance['IBV_Balance'].values[0]>200),
                            int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].values.mean())>200),
             int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].mean())>(df_balance[df_balance.within_last_7day==False]['IBV_Balance'].mean())) ]], 
              columns = ['account_id','BALANCE_MEAN','BALANCE_SD','BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                         'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D'])
        
        # # Red Zone features added by Coral: loan, payroll, deposit, total credit, total debit
        f18 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Debit']>60)].groupby('account_id').agg({'IBV_Debit':['count','sum']}).reset_index()
        f18.columns = ['account_id', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT']
        
        f19 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Credit']>60)].groupby('account_id').agg({'IBV_Credit':['count','sum']}).reset_index()
        f19.columns = ['account_id', 'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT']
        
        f20 = df_ibv[(df_ibv['regularPayroll']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f20.columns = ['account_id','PAYROLL_AMOUNT']
        

        f21 = df_ibv[(df_ibv['isDeposit']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f21.columns = ['account_id','DEPOSIT_AMOUNT']
        f22 = df_ibv.groupby('account_id')[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f22.columns = ['account_id','TOTAL_CREDIT','TOTAL_DEBIT']
        
        # # Alert indicators added by Coral: stop payment, nsf, overdraft, return
        stop_keywords = ['e-transfer stop', 'stop payt','stop payment','stop fee','stop pmt','stoppa','stop pymt']
        nsf_keywords = [r"\bnsf", 'NON-SUFFICIENT FUNDS','Insufficient fund','NON SUFFICIENT','Returned Item Fee']
        overdraft_keywords = ['Overdraft', 'Over limit','OD FEE','OD PROTECTION','OD HANDL']
        return_keywords = ['returned', 'RETURN FEE','Return of','EFT.*Return','EFT.*Reversal','rtn.*eft']
        gambling_keywords = ['payper', 'paybilt',' gigadat', 'playnow', 'bet.*river','betmgm','betty.*gaming',
                            'bally.*bet', 'pointsbet', 'bet365', 'betway', 'betano', 'thescore.*bet', 'northstar.*bet',
                            'lotto', 'playalberta', 'playtime', 'casino', 'ilixium.*casin']
        
        df_ibv['STOP_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(stop_keywords), case=False)) & (df_ibv['IBV_Credit']>0), 'STOP_Incident'] = 1
        df_ibv['NSF_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(nsf_keywords), case=False)), 'NSF_Incident'] = 1
        df_ibv['OVERDRAFT_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(overdraft_keywords), case=False)), 'OVERDRAFT_Incident'] = 1
        df_ibv['RETURN_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(return_keywords), case=False)), 'RETURN_Incident'] = 1
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'Gambling_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'IBV_Debit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'EI_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'IBV_Credit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'bree'), case=False)), 'BREE_Incident'] = 1
        
        
        f23 = df_ibv.groupby('account_id').agg(NUM_STOP_PMT = ('STOP_Incident','sum'),
                                               NUM_NSF = ('NSF_Incident','sum'),
                                               NUM_OD = ('OVERDRAFT_Incident','sum'),
                                               NUM_RETURN_PMT = ('RETURN_Incident','sum'),
                                               NUM_GAMBLING_PMT = ('Gambling_Incident', 'count'),
                                               AMT_GAMBLING_PMT = ('Gambling_Incident', 'sum'),
                                               NUM_EI_PMT = ('EI_Incident', 'count'),
                                               AMT_EI_PMT = ('EI_Incident', 'sum'),
                                               NUM_Bree = ('BREE_Incident', 'count')
                                              ).reset_index()

        
        ibv_modelinput = f1.copy()
        for i in [f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23]:
            ibv_modelinput = ibv_modelinput.merge(i, on='account_id',how='left')

        # ## Step 3: DataCleaning after feature generation
        stepname = '4 DataCleaning after Feature Gen'
        ibv_modelinput = ibv_modelinput.fillna(value=0).replace(np.inf, 100).replace(-np.inf, -100)
        # ## Step 4: IBV Model Scoring and IBVBand Assignment based on IBVScore
        ibv_features = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
                'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
                'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
                'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
                'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
                'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
                'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
                'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
                'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
                'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT',
                'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']
        
        # Load Model
        stepname = '5 Model Loading & Scoring'
        scaler = joblib.load(scalerfilepath)
        clf_ibv = joblib.load(modelfilepath)

        # Get IBV Score (output 1)
        standardized_data = ibv_modelinput[ibv_features].copy()
        # return ibv_modelinput, scaler, standardized_data,clf_ibv
        standardized_data[ibv_features] = scaler.transform(standardized_data)
        ibv_modelinput['IBVScore'] = 1000 - np.round(clf_ibv.predict_proba(standardized_data)[0][1]*1000,0).astype(int)

        # Get IBV Band (output 2)
        ibv_modelinput['IBVBand'] = np.where(ibv_modelinput['IBVScore']< 452,1, 
                                             np.where(ibv_modelinput['IBVScore']< 621,2, 
                                            np.where(ibv_modelinput['IBVScore'] < 700,3,
                                            np.where(ibv_modelinput['IBVScore'] < 753,4,
                                            5))))
        IBVScore = int(ibv_modelinput.IBVScore.values[0])
        IBVBand = int(ibv_modelinput.IBVBand.values[0])
    if 0:    
    # except:
        try:
            bankaccountid = df_ibv['account_id'].values[0]
            ndbaccountid = json_temp['NDB']['accountnumber']
            logging.exception('AccountID: ' + str(bankaccountid) + '; NDBAccount: ' + str(ndbaccountid) + '; ErrorInStep: ' + stepname )
        except: # If plaid file received is empty: account_id will be 0, or modified to Other value 
            logging.exception('AccountID: ' + 'Not Available' + '; ErrorInStep: ' + stepname )
        finally:
            return "{\"ModelScore\":"+str(0)+ ",\"IBVBand\":" + str(0)+"}"

 
    return "{\"ModelScore\":"+str(IBVScore)+ ",\"IBVBand\":" + str(IBVBand)+"}"

In [13]:
import json

data = 'IBV_786.json'
modelfilepath = "IBV_V3_TestBed.pkl"
scalerfilepath = 'IBV_SCALER_V3_TestBed.pkl'
# ibvmodel(data, modelfilepath, scalerfilepath)

In [14]:
joblib.load(modelfilepath)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)

In [17]:
jsonstr = 'IBV_786.json'
stepname = '1 Loading Json'
modelfilepath = "IBV_V3_TestBed.pkl"
scalerfilepath = 'standardscaler_V3.pkl'
if True:
# try:
    # Load and Read the JSON file  #### This is for testing changes made different from production
    # json_temp = json.loads(jsonstr)
    with open(jsonstr) as json_file:
        json_temp = json.load(json_file)
        

    # Load Json file's historical transaction part
    df_ibv = pd.read_json(json.dumps(json_temp['Historical_Transactions']))

    # Data Cleaning
    stepname = '2 Data Cleaning'
    df_ibv['IBV_Credit'] = np.where(df_ibv['amount']<0, -1*df_ibv['amount'],0)
    df_ibv['IBV_Debit'] = np.where(df_ibv['amount']>=0, df_ibv['amount'],0)
    df_ibv = df_ibv.rename(columns={'date':'IBV_Date','category':'IBV_BalCategory','name':'IBV_Description'})
    df_ibv['IBV_Date'] = pd.to_datetime(df_ibv['IBV_Date'])
    df_ibv['Year'] = df_ibv['IBV_Date'].apply(lambda x: x.year)
    df_ibv['Month'] = df_ibv['IBV_Date'].apply(lambda x: x.month)
    df_ibv = df_ibv[['account_id','IBV_Credit','IBV_Debit','IBV_Date','Year','Month','IBV_BalCategory','IBV_Description']]
    df_ibv['isLoan'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Service","Financial","Loans and Mortgages"],
                                        ["Travel and Transportation","Auto Loan"], ["Education","Student Loan"],
                                        ["Miscellaneous","Point of Sale Loan"],["Miscellaneous","Insolvency Loan"],
                                        ["Miscellaneous","Other Loan"],["Miscellaneous","Micro Loan"]] else 0)
    df_ibv['regularPayroll'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Transfer","Payroll"], ["Transfer","Payroll",'Benefits'],
                                ["Income","Pension"],["Income","Government"],["Income","Wages and Salary"]] else 0)
    df_ibv['isFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","ATM"],["Bank Fees","Cash Advance"],
                                            ["Bank Fees","Excess Activity"], ["Bank Fees","Foreign Transaction"],
                                            ["Bank Fees","Fraud Dispute"],["Bank Fees","Insufficient Funds"],
                                            ["Bank Fees","Late Payment"], ["Bank Fees","Overdraft"],
                                            ["Bank Fees","Wire Transfer"],["Bank Fees"],
                                            ["Fees and Charges","ATM Fees"], ["Fees and Charges","Service Fees"]]) else 0)
    df_ibv['isNSFFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","Insufficient Funds"],["Bank Fees","Overdraft"],
                                        ["Fees and Charges","Service Fees"]]) else 0)
    df_ibv['isDeposit'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Transfer","Deposit","ATM"],
                                            ["Transfer","Deposit","Check"],["Transfer","Deposit"], ["Miscellaneous Income","Deposit"]]) else 0)
    df_ibv['IBV_BalCategory'] = df_ibv['IBV_BalCategory'].apply(str)
    accountid = df_ibv['account_id'].values[0]

    # Load Json file's historical balance part
    primary_account_info = [i for i in json_temp['Historical_Balance']['report']['items'][0]['accounts'] 
                        if i['account_id']==df_ibv.account_id.values[0]]
    if primary_account_info == []:
        df_balance = pd.DataFrame( [[0.00, '2023-02-06', 'CAD' , None]], columns = ['IBV_Balance','date',
                                                    'iso_currency_code', 'unofficial_currency_code'] )
    else:
        df_balance = pd.DataFrame(primary_account_info[0]['historical_balances'])
        df_balance = df_balance.rename(columns={'current':'IBV_Balance'})
    df_balance.date = pd.to_datetime(df_balance.date)
    
    ### Make sure you change it!!!
    curdate = date.today()
    # curdate = df_balance['date'].max().date() # remove it before deployment
    
    df_balance['within_last_7day'] = df_balance.apply(lambda x: (curdate - x['date'].date()).days<=7, axis=1)

    ## Step 2: Feature Generation for IBV: to get the input of IBV_Model

    # Feature Generation

    # # Max History
    stepname = '3 Feature Generation'
    
    f1 = pd.DataFrame([[df_ibv.account_id.values[0],(df_ibv['IBV_Date'].max() - df_ibv['IBV_Date'].min()).days]],
                    columns = ['account_id','MaxHistory'])

    # # CREDIT_TO_DEBIT_RATIO_MEAN, CREDIT_TO_DEBIT_RATIO_SD
    f2 = df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()
    f2['CREDIT_TO_DEBIT_RATIO'] = f2['IBV_Credit'] / f2['IBV_Debit']
    f2 = f2.groupby('account_id').agg({'CREDIT_TO_DEBIT_RATIO':['mean','std']}).reset_index()
    f2.columns = ['account_id','CREDIT_TO_DEBIT_RATIO_MEAN','CREDIT_TO_DEBIT_RATIO_SD']

    # # DAILY_DEBIT_AMOUNT_MEAN, DAILY_INCOME_MEAN
    f3 = df_ibv[df_ibv.isLoan==0].groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()
    f3['IBV_Credit'] = f3['IBV_Credit'] / 30
    f3['IBV_Debit'] = f3['IBV_Debit'] / 30
    f3 = f3.groupby('account_id')[['IBV_Debit','IBV_Credit']].mean().reset_index()
    f3.columns = ['account_id','DAILY_DEBIT_AMOUNT_MEAN','DAILY_INCOME_MEAN']

    # # DAILY_INCOME_REGULAR_MEAN
    f4 = df_ibv[df_ibv.regularPayroll==1].groupby(['account_id','Year','Month'])['IBV_Credit'].sum().reset_index()
    f4['IBV_Credit'] = f4['IBV_Credit'] / 30
    f4 = f4.groupby('account_id').mean()['IBV_Credit'].reset_index()
    f4.columns = ['account_id','DAILY_INCOME_REGULAR_MEAN']

    # # DEBIT_AMOUNT_AVG, DEBIT_AMOUNT_SD, DEBIT_COUNT_AVG,DEBIT_COUNT_SD
    f5 = df_ibv.groupby(['account_id','Year','Month'])['IBV_Debit'].sum().reset_index()
    f5 = f5.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
    f5.columns = ['account_id','DEBIT_AMOUNT_MEAN','DEBIT_AMOUNT_SD']

    f6 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month'])['IBV_Debit'].count().reset_index()
    f6 = f6.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
    f6.columns = ['account_id','DEBIT_COUNT_MEAN','DEBIT_COUNT_SD']

    # # DEBIT_AMOUNT_Z, DEBIT_COUNT_Z
    f71 = df_ibv.groupby(['account_id','Year','Month'])['IBV_Debit'].sum().reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
    f71 = f71.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
    f71 = f71.merge(f5,on='account_id')
    f71['DEBIT_AMOUNT_Z'] = (f71['IBV_Debit'] - f71['DEBIT_AMOUNT_MEAN'])/f71['DEBIT_AMOUNT_SD']

    f72 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month'])['IBV_Debit'].count().reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
    f72 = f72.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
    f72 = f72.merge(f6,on='account_id')
    f72['DEBIT_COUNT_Z'] = (f72['IBV_Debit'] - f72['DEBIT_COUNT_MEAN'])/f72['DEBIT_COUNT_SD']

    f7 = f71[['account_id', 'DEBIT_AMOUNT_Z']].merge(f72[['account_id','DEBIT_COUNT_Z']], on='account_id', how='left')


    # # HIGHEST_PAY_DEPOSIT_MEAN, HIGHEST_PAY_FREQUENCY
    f81 = df_ibv.groupby(['account_id','IBV_BalCategory','Year','Month'])['IBV_Credit'].sum().reset_index()
    f81 = f81.groupby(['account_id','IBV_BalCategory'])['IBV_Credit'].mean().reset_index().sort_values(by=['account_id','IBV_Credit','IBV_BalCategory'],ascending=False)
    f82 = f81.groupby('account_id').first().reset_index()[['account_id','IBV_BalCategory']]
    f8 = f81.merge(f82, on =['account_id','IBV_BalCategory'], how='inner')[['account_id','IBV_Credit']]
    f8.columns = ['account_id','HIGHEST_PAY_DEPOSIT_MEAN']

    f9 = df_ibv[df_ibv.IBV_Credit>0].merge(f82, on =['account_id','IBV_BalCategory'], how='inner').groupby(['account_id','Year','Month'])['IBV_Credit'].count().reset_index()
    f9 = f9.groupby('account_id')['IBV_Credit'].mean().reset_index()[['account_id','IBV_Credit']]
    f9.columns = ['account_id','HIGHEST_PAY_FREQUENCY']

    # # INCOME_SOURCES_COUNT
    f10 = df_ibv[(df_ibv.IBV_Credit>0)&(df_ibv.IBV_BalCategory.isin(['income/investment_income','income/paycheck','income/bonus','income/government',
    'income','income/pension','income/child_support']))].groupby(['account_id','IBV_BalCategory']).size().reset_index()
    f10 = f10.groupby('account_id').size().reset_index()
    f10.columns = ['account_id','INCOME_SOURCES_COUNT']



    # # MONTH_INFLOW_MEAN, MONTH_INFLOW_SD, MONTH_OUTFLOW_MEAN, MONTH_OUTFLOW_SD
    f11 = df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()
    f11 = f11.groupby('account_id').agg({'IBV_Credit':['mean','std'],'IBV_Debit':['mean','std']}).reset_index()
    f11.columns = ['account_id','MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD']


    # #MONTHS_WITH_FEES_RATE
    f12_ = df_ibv.copy()
    f12 = f12_.groupby(['account_id','Year','Month'])['isFees'].sum().reset_index()
    f12['MONTHS_WITH_FEES_RATE'] = np.where(f12['isFees']>0,1.0,0.0)
    f12 = f12.groupby('account_id')['MONTHS_WITH_FEES_RATE'].mean().reset_index()[['account_id','MONTHS_WITH_FEES_RATE']]

    # # MONTHS_WITH_EMPLOYMENT_RATE
    f12_['Emp'] = (df_ibv['IBV_BalCategory']=='income/paycheck').astype(int) #!!!!!
    f13 = f12_.groupby(['account_id','Year','Month'])['Emp'].sum().reset_index()
    f13['MONTHS_WITH_EMPLOYMENT_RATE'] = np.where(f13['Emp']>0,1.0,0.0)
    f13 = f13.groupby('account_id').mean()['MONTHS_WITH_EMPLOYMENT_RATE'].reset_index()[['account_id','MONTHS_WITH_EMPLOYMENT_RATE']]

    # # NO_ACTIVITY_RATE
    f14 = df_ibv.groupby('account_id')['IBV_Date'].nunique().reset_index()
    f14 = f14.merge(f1, on='account_id')[['account_id','IBV_Date','MaxHistory']]
    f14['NO_ACTIVITY_RATE'] = f14['IBV_Date'] / f14['MaxHistory']
    f14 = f14[['account_id','NO_ACTIVITY_RATE']]

    # # OD_AND_NSF_FEES_DAILY
    f15 = df_ibv[df_ibv.isNSFFees==1].groupby('account_id')['IBV_Debit'].sum().reset_index()
    f15 = f15.merge(f1, on='account_id')[['account_id','IBV_Debit','MaxHistory']]
    f15['OD_AND_NSF_FEES_DAILY'] = f15['IBV_Debit'] / f15['MaxHistory']
    f15 = f15[['account_id','OD_AND_NSF_FEES_DAILY']]

    # # RECURRENT_COUNT, RECURRENT_RATE
    f16 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id', 'Year', 'Month', 'IBV_Debit', 'IBV_BalCategory']).size()
    f16 = f16.groupby(['account_id','IBV_Debit','IBV_BalCategory']).size().reset_index()
    f16.columns = ['account_id','IBV_Debit', 'IBV_BalCategory', 'CNT']
    f16 = f16[(f16['CNT']>=5)&(f16['IBV_Debit']>=10)]
    f16 = f16.groupby('account_id').agg({'IBV_BalCategory':'nunique','IBV_Debit':'sum'}).reset_index()
    f16.columns = ['account_id','RECURRENT_COUNT','RECURRING_RATE']
    
    f17 = pd.DataFrame([[accountid, df_balance['IBV_Balance'].mean(), df_balance['IBV_Balance'].std(),
                        (df_balance['IBV_Balance']>200).mean(), int(df_balance['IBV_Balance'].values[0]>200),
                        int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].values.mean())>200),
            int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].mean())>(df_balance[df_balance.within_last_7day==False]['IBV_Balance'].mean())) ]], 
            columns = ['account_id','BALANCE_MEAN','BALANCE_SD','BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                        'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D'])
    
    # # Red Zone features added by Coral: loan, payroll, deposit, total credit, total debit
    f18 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Debit']>60)].groupby('account_id').agg({'IBV_Debit':['count','sum']}).reset_index()
    f18.columns = ['account_id', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT']
    
    f19 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Credit']>60)].groupby('account_id').agg({'IBV_Credit':['count','sum']}).reset_index()
    f19.columns = ['account_id', 'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT']
    
    f20 = df_ibv[(df_ibv['regularPayroll']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
    f20.columns = ['account_id','PAYROLL_AMOUNT']
    

    f21 = df_ibv[(df_ibv['isDeposit']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
    f21.columns = ['account_id','DEPOSIT_AMOUNT']
    f22 = df_ibv.groupby('account_id')[['IBV_Credit','IBV_Debit']].sum().reset_index()
    f22.columns = ['account_id','TOTAL_CREDIT','TOTAL_DEBIT']
    
    # # Alert indicators added by Coral: stop payment, nsf, overdraft, return
    stop_keywords = ['e-transfer stop', 'stop payt','stop payment','stop fee','stop pmt','stoppa','stop pymt']
    nsf_keywords = [r"\bnsf", 'NON-SUFFICIENT FUNDS','Insufficient fund','NON SUFFICIENT','Returned Item Fee']
    overdraft_keywords = ['Overdraft', 'Over limit','OD FEE','OD PROTECTION','OD HANDL']
    return_keywords = ['returned', 'RETURN FEE','Return of','EFT.*Return','EFT.*Reversal','rtn.*eft']
    gambling_keywords = ['payper', 'paybilt',' gigadat', 'playnow', 'bet.*river','betmgm','betty.*gaming',
                        'bally.*bet', 'pointsbet', 'bet365', 'betway', 'betano', 'thescore.*bet', 'northstar.*bet',
                        'lotto', 'playalberta', 'playtime', 'casino', 'ilixium.*casin']
    
    df_ibv['STOP_Incident'] = 0
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(stop_keywords), case=False)) & (df_ibv['IBV_Credit']>0), 'STOP_Incident'] = 1
    df_ibv['NSF_Incident'] = 0
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(nsf_keywords), case=False)), 'NSF_Incident'] = 1
    df_ibv['OVERDRAFT_Incident'] = 0
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(overdraft_keywords), case=False)), 'OVERDRAFT_Incident'] = 1
    df_ibv['RETURN_Incident'] = 0
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(return_keywords), case=False)), 'RETURN_Incident'] = 1
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'Gambling_Incident'] = \
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'IBV_Debit'].values
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'EI_Incident'] = \
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'IBV_Credit'].values
    df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'bree'), case=False)), 'BREE_Incident'] = 1
    
    
    f23 = df_ibv.groupby('account_id').agg(NUM_STOP_PMT = ('STOP_Incident','sum'),
                                            NUM_NSF = ('NSF_Incident','sum'),
                                            NUM_OD = ('OVERDRAFT_Incident','sum'),
                                            NUM_RETURN_PMT = ('RETURN_Incident','sum'),
                                            NUM_GAMBLING_PMT = ('Gambling_Incident', 'count'),
                                            AMT_GAMBLING_PMT = ('Gambling_Incident', 'sum'),
                                            NUM_EI_PMT = ('EI_Incident', 'count'),
                                            AMT_EI_PMT = ('EI_Incident', 'sum'),
                                            NUM_Bree = ('BREE_Incident', 'count')
                                            ).reset_index()

    
    ibv_modelinput = f1.copy()
    for i in [f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23]:
        ibv_modelinput = ibv_modelinput.merge(i, on='account_id',how='left')

    # ## Step 3: DataCleaning after feature generation
    stepname = '4 DataCleaning after Feature Gen'
    ibv_modelinput = ibv_modelinput.fillna(value=0).replace(np.inf, 100).replace(-np.inf, -100)
    # ## Step 4: IBV Model Scoring and IBVBand Assignment based on IBVScore
    ibv_features = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
            'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
            'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
            'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
            'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
            'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
            'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
            'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
            'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
            'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
            'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
            'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT',
            'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']
    
    # Load Model
    stepname = '5 Model Loading & Scoring'
    scaler = joblib.load(scalerfilepath)
    clf_ibv = joblib.load(modelfilepath)

    # Get IBV Score (output 1)
    standardized_data = ibv_modelinput[ibv_features].copy()
    # return ibv_modelinput, scaler, standardized_data,clf_ibv
    standardized_data[ibv_features] = scaler.transform(standardized_data)
    ibv_modelinput['IBVScore'] = 1000 - np.round(clf_ibv.predict_proba(standardized_data)[0][1]*1000,0).astype(int)

    # Get IBV Band (output 2)
    ibv_modelinput['IBVBand'] = np.where(ibv_modelinput['IBVScore']< 452,1, 
                                            np.where(ibv_modelinput['IBVScore']< 621,2, 
                                        np.where(ibv_modelinput['IBVScore'] < 700,3,
                                        np.where(ibv_modelinput['IBVScore'] < 753,4,
                                        5))))
    IBVScore = int(ibv_modelinput.IBVScore.values[0])
    IBVBand = int(ibv_modelinput.IBVBand.values[0])
if 0:    
# except:
    try:
        bankaccountid = df_ibv['account_id'].values[0]
        ndbaccountid = json_temp['NDB']['accountnumber']
        logging.exception('AccountID: ' + str(bankaccountid) + '; NDBAccount: ' + str(ndbaccountid) + '; ErrorInStep: ' + stepname )
    except: # If plaid file received is empty: account_id will be 0, or modified to Other value 
        logging.exception('AccountID: ' + 'Not Available' + '; ErrorInStep: ' + stepname )
    finally:
        print(1) # return "{\"ModelScore\":"+str(0)+ ",\"IBVBand\":" + str(0)+"}"


In [18]:
print(IBVScore)

import os
os.cpu_count()

853


2

In [5]:
df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()

,account_id,Year,Month,IBV_Credit,IBV_Debit
0,Kzag3vgegrIkaxVb6654UB1MeX6R35CyOMBEn,2023,2,1989.37,1854.36
1,Kzag3vgegrIkaxVb6654UB1MeX6R35CyOMBEn,2023,3,5917.92,5799.26
2,Kzag3vgegrIkaxVb6654UB1MeX6R35CyOMBEn,2023,4,6523.41,6318.75
3,Kzag3vgegrIkaxVb6654UB1MeX6R35CyOMBEn,2023,5,3572.04,3473.28


In [26]:
ls

 IBVModel_WarningErrorLog_2023.log*   ibvv3-copy.ipynb.amltmp*
 IBVV3-KernelV16g.ipynb*              ibvv3-kernelv16g.ipynb.amltmp*
 IBVV3.ipynb*                         ibvv3.ipynb.amltmp*
 IBV_786.json*                       'inputJSON for PlaidUDW_v1.json'*
 IBV_787.json*                        standardscaler_V3.pkl*
 IBV_SCALER_V3_TestBed.pkl*           trainingforstandardization.csv*
 IBV_V3_TestBed.pkl*                  traininginputdata.csv*
 ibv_input_data.csv*                  trainingoutputdata.csv*
 ibvv#.ipynb.amltmp*


In [6]:
# Actual Deveopds code: 1) change try except to if true and if false


from datetime import datetime, date
import pandas as pd
import numpy as np
import statsmodels.api as sm
import joblib
import json
import sys
import logging

# example: jsonstr = "IBV_1051.json"
# example: modelfilepath = "IBV_V3_TestBed.pkl"
# example: scalerfilepath = 'standardscaler_V3.pkl'
# example use case: ibvmodel(jsonstr, modelfilepath, scalerfilepath)

def ibvmodel(jsonstr, modelfilepath, scalerfilepath):
    
    logging.basicConfig(filename="IBVModel_WarningErrorLog_2023.log", level=logging.WARNING, 
            format="%(asctime)s - %(levelname)s - %(message)s")
    stepname = '1 Loading Json'

    if True:
        # Load and Read the JSON file
        json_temp = json.loads(jsonstr)
        # with open(jsonstr) as json_file:
        #     json_temp = json.load(json_file)
        # Load Json file's historical transaction part
        df_ibv = pd.read_json(json.dumps(json_temp['Historical_Transactions']))

        # Data Cleaning
        stepname = '2 Data Cleaning'
        df_ibv['IBV_Credit'] = np.where(df_ibv['amount']<0, -1*df_ibv['amount'],0)
        df_ibv['IBV_Debit'] = np.where(df_ibv['amount']>=0, df_ibv['amount'],0)
        df_ibv = df_ibv.rename(columns={'date':'IBV_Date','category':'IBV_BalCategory','name':'IBV_Description'})
        df_ibv['IBV_Date'] = pd.to_datetime(df_ibv['IBV_Date'])
        df_ibv['Year'] = df_ibv['IBV_Date'].apply(lambda x: x.year)
        df_ibv['Month'] = df_ibv['IBV_Date'].apply(lambda x: x.month)
        df_ibv = df_ibv[['account_id','IBV_Credit','IBV_Debit','IBV_Date','Year','Month','IBV_BalCategory','IBV_Description']]
        df_ibv['isLoan'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Service","Financial","Loans and Mortgages"],
                                            ["Travel and Transportation","Auto Loan"], ["Education","Student Loan"],
                                            ["Miscellaneous","Point of Sale Loan"],["Miscellaneous","Insolvency Loan"],
                                            ["Miscellaneous","Other Loan"],["Miscellaneous","Micro Loan"]] else 0)
        df_ibv['regularPayroll'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Transfer","Payroll"], ["Transfer","Payroll",'Benefits'],
                                    ["Income","Pension"],["Income","Government"],["Income","Wages and Salary"]] else 0)
        df_ibv['isFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","ATM"],["Bank Fees","Cash Advance"],
                                                ["Bank Fees","Excess Activity"], ["Bank Fees","Foreign Transaction"],
                                                ["Bank Fees","Fraud Dispute"],["Bank Fees","Insufficient Funds"],
                                                ["Bank Fees","Late Payment"], ["Bank Fees","Overdraft"],
                                                ["Bank Fees","Wire Transfer"],["Bank Fees"],
                                                ["Fees and Charges","ATM Fees"], ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isNSFFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","Insufficient Funds"],["Bank Fees","Overdraft"],
                                            ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isDeposit'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Transfer","Deposit","ATM"],
                                                ["Transfer","Deposit","Check"],["Transfer","Deposit"], ["Miscellaneous Income","Deposit"]]) else 0)
        df_ibv['IBV_BalCategory'] = df_ibv['IBV_BalCategory'].apply(str)
        accountid = df_ibv['account_id'].values[0]

        # Load Json file's historical balance part
        primary_account_info = [i for i in json_temp['Historical_Balance']['report']['items'][0]['accounts'] 
                            if i['account_id']==df_ibv.account_id.values[0]]
        if primary_account_info == []:
            df_balance = pd.DataFrame( [[0.00, '2023-02-06', 'CAD' , None]], columns = ['IBV_Balance','date',
                                                        'iso_currency_code', 'unofficial_currency_code'] )
        else:
            df_balance = pd.DataFrame(primary_account_info[0]['historical_balances'])
            df_balance = df_balance.rename(columns={'current':'IBV_Balance'})
        df_balance.date = pd.to_datetime(df_balance.date)
        
        ### Make sure you change it!!!
        curdate = date.today()
        # curdate = df_balance['date'].max().date() # remove it before deployment
        
        df_balance['within_last_7day'] = df_balance.apply(lambda x: (curdate - x['date'].date()).days<=7, axis=1)

        ## Step 2: Feature Generation for IBV: to get the input of IBV_Model

        # Feature Generation

        # # Max History
        stepname = '3 Feature Generation'
    
        f1 = pd.DataFrame([[df_ibv.account_id.values[0],(df_ibv['IBV_Date'].max() - df_ibv['IBV_Date'].min()).days]],
                        columns = ['account_id','MaxHistory'])

        # # CREDIT_TO_DEBIT_RATIO_MEAN, CREDIT_TO_DEBIT_RATIO_SD
        f2 = df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f2['CREDIT_TO_DEBIT_RATIO'] = f2['IBV_Credit'] / f2['IBV_Debit']
        f2 = f2.groupby('account_id').agg({'CREDIT_TO_DEBIT_RATIO':['mean','std']}).reset_index()
        f2.columns = ['account_id','CREDIT_TO_DEBIT_RATIO_MEAN','CREDIT_TO_DEBIT_RATIO_SD']

        # # DAILY_DEBIT_AMOUNT_MEAN, DAILY_INCOME_MEAN
        f3 = df_ibv[df_ibv.isLoan==0].groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f3['IBV_Credit'] = f3['IBV_Credit'] / 30
        f3['IBV_Debit'] = f3['IBV_Debit'] / 30
        f3 = f3.groupby('account_id')[['IBV_Debit','IBV_Credit']].mean().reset_index()
        f3.columns = ['account_id','DAILY_DEBIT_AMOUNT_MEAN','DAILY_INCOME_MEAN']

        # # DAILY_INCOME_REGULAR_MEAN
        f4 = df_ibv[df_ibv.regularPayroll==1].groupby(['account_id','Year','Month'])['IBV_Credit'].sum().reset_index()
        f4['IBV_Credit'] = f4['IBV_Credit'] / 30
        f4 = f4.groupby('account_id').mean()['IBV_Credit'].reset_index()
        f4.columns = ['account_id','DAILY_INCOME_REGULAR_MEAN']

        # # DEBIT_AMOUNT_AVG, DEBIT_AMOUNT_SD, DEBIT_COUNT_AVG,DEBIT_COUNT_SD
        f5 = df_ibv.groupby(['account_id','Year','Month'])['IBV_Debit'].sum().reset_index()
        f5 = f5.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f5.columns = ['account_id','DEBIT_AMOUNT_MEAN','DEBIT_AMOUNT_SD']

        f6 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month'])['IBV_Debit'].count().reset_index()
        f6 = f6.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f6.columns = ['account_id','DEBIT_COUNT_MEAN','DEBIT_COUNT_SD']

        # # DEBIT_AMOUNT_Z, DEBIT_COUNT_Z
        f71 = df_ibv.groupby(['account_id','Year','Month'])['IBV_Debit'].sum().reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f71 = f71.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f71 = f71.merge(f5,on='account_id')
        f71['DEBIT_AMOUNT_Z'] = (f71['IBV_Debit'] - f71['DEBIT_AMOUNT_MEAN'])/f71['DEBIT_AMOUNT_SD']

        f72 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month'])['IBV_Debit'].count().reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f72 = f72.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f72 = f72.merge(f6,on='account_id')
        f72['DEBIT_COUNT_Z'] = (f72['IBV_Debit'] - f72['DEBIT_COUNT_MEAN'])/f72['DEBIT_COUNT_SD']

        f7 = f71[['account_id', 'DEBIT_AMOUNT_Z']].merge(f72[['account_id','DEBIT_COUNT_Z']], on='account_id', how='left')

        # # HIGHEST_PAY_DEPOSIT_MEAN, HIGHEST_PAY_FREQUENCY
        f81 = df_ibv.groupby(['account_id','IBV_BalCategory','Year','Month'])['IBV_Credit'].sum().reset_index()
        f81 = f81.groupby(['account_id','IBV_BalCategory'])['IBV_Credit'].mean().reset_index().sort_values(by=['account_id','IBV_Credit','IBV_BalCategory'],ascending=False)
        f82 = f81.groupby('account_id').first().reset_index()[['account_id','IBV_BalCategory']]
        f8 = f81.merge(f82, on =['account_id','IBV_BalCategory'], how='inner')[['account_id','IBV_Credit']]
        f8.columns = ['account_id','HIGHEST_PAY_DEPOSIT_MEAN']

        f9 = df_ibv[df_ibv.IBV_Credit>0].merge(f82, on =['account_id','IBV_BalCategory'], how='inner').groupby(['account_id','Year','Month'])['IBV_Credit'].count().reset_index()
        f9 = f9.groupby('account_id')['IBV_Credit'].mean().reset_index()[['account_id','IBV_Credit']]
        f9.columns = ['account_id','HIGHEST_PAY_FREQUENCY']

        # # INCOME_SOURCES_COUNT
        f10 = df_ibv[(df_ibv.IBV_Credit>0)&(df_ibv.IBV_BalCategory.isin(['income/investment_income','income/paycheck','income/bonus','income/government',
        'income','income/pension','income/child_support']))].groupby(['account_id','IBV_BalCategory']).size().reset_index()
        f10 = f10.groupby('account_id').size().reset_index()
        f10.columns = ['account_id','INCOME_SOURCES_COUNT']



        # # MONTH_INFLOW_MEAN, MONTH_INFLOW_SD, MONTH_OUTFLOW_MEAN, MONTH_OUTFLOW_SD
        f11 = df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f11 = f11.groupby('account_id').agg({'IBV_Credit':['mean','std'],'IBV_Debit':['mean','std']}).reset_index()
        f11.columns = ['account_id','MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD']


        # #MONTHS_WITH_FEES_RATE
        f12_ = df_ibv.copy()
        f12 = f12_.groupby(['account_id','Year','Month'])['isFees'].sum().reset_index()
        f12['MONTHS_WITH_FEES_RATE'] = np.where(f12['isFees']>0,1.0,0.0)
        f12 = f12.groupby('account_id')['MONTHS_WITH_FEES_RATE'].mean().reset_index()[['account_id','MONTHS_WITH_FEES_RATE']]

        # # MONTHS_WITH_EMPLOYMENT_RATE
        f12_['Emp'] = (df_ibv['IBV_BalCategory']=='income/paycheck').astype(int) #!!!!!
        f13 = f12_.groupby(['account_id','Year','Month'])['Emp'].sum().reset_index()
        f13['MONTHS_WITH_EMPLOYMENT_RATE'] = np.where(f13['Emp']>0,1.0,0.0)
        f13 = f13.groupby('account_id').mean()['MONTHS_WITH_EMPLOYMENT_RATE'].reset_index()[['account_id','MONTHS_WITH_EMPLOYMENT_RATE']]

        # # NO_ACTIVITY_RATE
        f14 = df_ibv.groupby('account_id')['IBV_Date'].nunique().reset_index()
        f14 = f14.merge(f1, on='account_id')[['account_id','IBV_Date','MaxHistory']]
        f14['NO_ACTIVITY_RATE'] = f14['IBV_Date'] / f14['MaxHistory']
        f14 = f14[['account_id','NO_ACTIVITY_RATE']]

        # # OD_AND_NSF_FEES_DAILY
        f15 = df_ibv[df_ibv.isNSFFees==1].groupby('account_id')['IBV_Debit'].sum().reset_index()
        f15 = f15.merge(f1, on='account_id')[['account_id','IBV_Debit','MaxHistory']]
        f15['OD_AND_NSF_FEES_DAILY'] = f15['IBV_Debit'] / f15['MaxHistory']
        f15 = f15[['account_id','OD_AND_NSF_FEES_DAILY']]

        # # RECURRENT_COUNT, RECURRENT_RATE
        f16 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id', 'Year', 'Month', 'IBV_Debit', 'IBV_BalCategory']).size()
        f16 = f16.groupby(['account_id','IBV_Debit','IBV_BalCategory']).size().reset_index()
        f16.columns = ['account_id','IBV_Debit', 'IBV_BalCategory', 'CNT']
        f16 = f16[(f16['CNT']>=5)&(f16['IBV_Debit']>=10)]
        f16 = f16.groupby('account_id').agg({'IBV_BalCategory':'nunique','IBV_Debit':'sum'}).reset_index()
        f16.columns = ['account_id','RECURRENT_COUNT','RECURRING_RATE']
        
        f17 = pd.DataFrame([[accountid, df_balance['IBV_Balance'].mean(), df_balance['IBV_Balance'].std(),
                            (df_balance['IBV_Balance']>200).mean(), int(df_balance['IBV_Balance'].values[0]>200),
                            int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].values.mean())>200),
                int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].mean())>(df_balance[df_balance.within_last_7day==False]['IBV_Balance'].mean())) ]], 
                columns = ['account_id','BALANCE_MEAN','BALANCE_SD','BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                            'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D'])
        
        # # Red Zone features added by Coral: loan, payroll, deposit, total credit, total debit
        f18 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Debit']>60)].groupby('account_id').agg({'IBV_Debit':['count','sum']}).reset_index()
        f18.columns = ['account_id', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT']
        
        f19 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Credit']>60)].groupby('account_id').agg({'IBV_Credit':['count','sum']}).reset_index()
        f19.columns = ['account_id', 'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT']
        
        f20 = df_ibv[(df_ibv['regularPayroll']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f20.columns = ['account_id','PAYROLL_AMOUNT']
        

        f21 = df_ibv[(df_ibv['isDeposit']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f21.columns = ['account_id','DEPOSIT_AMOUNT']
        f22 = df_ibv.groupby('account_id')[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f22.columns = ['account_id','TOTAL_CREDIT','TOTAL_DEBIT']
    
        
        # # Alert indicators added by Coral: stop payment, nsf, overdraft, return
        stop_keywords = ['e-transfer stop', 'stop payt','stop payment','stop fee','stop pmt','stoppa','stop pymt']
        nsf_keywords = [r"\bnsf", 'NON-SUFFICIENT FUNDS','Insufficient fund','NON SUFFICIENT','Returned Item Fee']
        overdraft_keywords = ['Overdraft', 'Over limit','OD FEE','OD PROTECTION','OD HANDL']
        return_keywords = ['returned', 'RETURN FEE','Return of','EFT.*Return','EFT.*Reversal','rtn.*eft']
        gambling_keywords = ['payper', 'paybilt',' gigadat', 'playnow', 'bet.*river','betmgm','betty.*gaming',
                            'bally.*bet', 'pointsbet', 'bet365', 'betway', 'betano', 'thescore.*bet', 'northstar.*bet',
                            'lotto', 'playalberta', 'playtime', 'casino', 'ilixium.*casin']
        
        df_ibv['STOP_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(stop_keywords), case=False)) & (df_ibv['IBV_Credit']>0), 'STOP_Incident'] = 1
        df_ibv['NSF_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(nsf_keywords), case=False)), 'NSF_Incident'] = 1
        df_ibv['OVERDRAFT_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(overdraft_keywords), case=False)), 'OVERDRAFT_Incident'] = 1
        df_ibv['RETURN_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(return_keywords), case=False)), 'RETURN_Incident'] = 1
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'Gambling_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'IBV_Debit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'EI_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'IBV_Credit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'bree'), case=False)), 'BREE_Incident'] = 1
        
        
        f23 = df_ibv.groupby('account_id').agg(NUM_STOP_PMT = ('STOP_Incident','sum'),
                                               NUM_NSF = ('NSF_Incident','sum'),
                                               NUM_OD = ('OVERDRAFT_Incident','sum'),
                                               NUM_RETURN_PMT = ('RETURN_Incident','sum'),
                                               NUM_GAMBLING_PMT = ('Gambling_Incident', 'count'),
                                               AMT_GAMBLING_PMT = ('Gambling_Incident', 'sum'),
                                               NUM_EI_PMT = ('EI_Incident', 'count'),
                                               AMT_EI_PMT = ('EI_Incident', 'sum'),
                                               NUM_Bree = ('BREE_Incident', 'count')
                                              ).reset_index()

        
        ibv_modelinput = f1.copy()
        for i in [f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23]:
            ibv_modelinput = ibv_modelinput.merge(i, on='account_id',how='left')

        # ## Step 3: DataCleaning after feature generation
        stepname = '4 DataCleaning after Feature Gen'
        ibv_modelinput = ibv_modelinput.fillna(value=0).replace(np.inf, 100).replace(-np.inf, -100)
        # ## Step 4: IBV Model Scoring and IBVBand Assignment based on IBVScore
        ibv_features = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
                'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
                'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
                'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
                'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
                'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
                'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
                'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
                'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
                'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT',
                'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']
        
        # Load Model
        stepname = '5 Model Loading & Scoring'
        scaler = joblib.load(scalerfilepath)
        clf_ibv = joblib.load(modelfilepath)

        # Get IBV Score (output 1)
        standardized_data = ibv_modelinput[ibv_features].copy()
        # return ibv_modelinput, scaler, standardized_data,clf_ibv
        standardized_data[ibv_features] = scaler.transform(standardized_data)
        ibv_modelinput['IBVScore'] = 1000 - np.round(clf_ibv.predict_proba(standardized_data)[0][1]*1000,0).astype(int)

        # Get IBV Band (output 2)
        ibv_modelinput['IBVBand'] = np.where(ibv_modelinput['IBVScore']< 452,1, 
                                             np.where(ibv_modelinput['IBVScore']< 621,2, 
                                            np.where(ibv_modelinput['IBVScore'] < 700,3,
                                            np.where(ibv_modelinput['IBVScore'] < 753,4,
                                            5))))
        IBVScore = int(ibv_modelinput.IBVScore.values[0])
        IBVBand = int(ibv_modelinput.IBVBand.values[0])


    if False:
        try:
            bankaccountid = df_ibv['account_id'].values[0]
            ndbaccountid = json_temp['NDB']['accountnumber']
            logging.exception('AccountID: ' + str(bankaccountid) + '; NDBAccount: ' + str(ndbaccountid) + '; ErrorInStep: ' + stepname )
        except: # If plaid file received is empty: account_id will be 0, or modified to Other value 
            logging.exception('AccountID: ' + 'Not Available' + '; ErrorInStep: ' + stepname )
        finally:
            return "{\"ModelScore\":"+str(0)+ ",\"IBVBand\":" + str(0)+"}"

 
    return "{\"ModelScore\":"+str(IBVScore)+ ",\"IBVBand\":" + str(IBVBand)+"}"

In [7]:
data = 'IBV_786.json'

with open(data, 'r') as json_file:
    data = json_file.read()
modelfilepath = "IBV_V3_TestBed.pkl"
scalerfilepath = 'standardscaler_V3.pkl'
ibvmodel(data, modelfilepath, scalerfilepath)

'{"ModelScore":853,"IBVBand":5}'

In [6]:
type(data)

str

In [8]:
import json
data = 'inputJSON for PlaidUDW_v1.json'
with open(data, 'r') as json_file:
    data = json_file.read()
modelfilepath = "IBV_V3_TestBed.pkl"
scalerfilepath = 'standardscaler_V3.pkl'
ibvmodel(data, modelfilepath, scalerfilepath)

'{"ModelScore":530,"IBVBand":2}'

In [9]:
data = 'inputJSON for PlaidUDW_v1 (3).json'
with open(data, 'r') as json_file:
    data = json_file.read()
modelfilepath = "IBV_V3_TestBed.pkl"
scalerfilepath = 'standardscaler_V3.pkl'
ibvmodel(data, modelfilepath, scalerfilepath)

'{"ModelScore":976,"IBVBand":5}'

In [12]:


lst = [i for i in os.listdir('TestData') if i.endswith('json')]


for i in lst:
    data = 'TestData/'+ str(i)
    with open(data, 'r') as json_file:
        data = json_file.read()
    modelfilepath = "IBV_V3_TestBed.pkl"
    scalerfilepath = 'standardscaler_V3.pkl'
    print(i, ibvmodel(data, modelfilepath, scalerfilepath))

IBV_56617.json {"ModelScore":959,"IBVBand":5}
IBV_56623.json {"ModelScore":959,"IBVBand":5}
IBV_56625.json {"ModelScore":971,"IBVBand":5}
IBV_56637.json {"ModelScore":931,"IBVBand":5}
IBV_56669.json {"ModelScore":979,"IBVBand":5}
IBV_56686.json {"ModelScore":974,"IBVBand":5}
IBV_56701.json {"ModelScore":865,"IBVBand":5}
IBV_56731.json {"ModelScore":869,"IBVBand":5}
IBV_56877.json {"ModelScore":983,"IBVBand":5}
IBV_56878.json {"ModelScore":940,"IBVBand":5}
